In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip3 install text2num
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=fede874aad94f4e08bc15a4614132c40665939782c64dea40da85f10cdb37348
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
!pip install word2number

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5566 sha256=1f9dcd5741b7ea27e7f905a0ffde6b5ef19bb3123b880534bff8d7b721b61555
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [4]:
from word2number import w2n

In [ ]:
##sympy setup
!git clone https://github.com/sympy/sympy.git
import os
os.chdir('sympy/')
os.getcwdb()
!python -m pip install -e .

In [5]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#df=pd.read_pickle("/content/drive/MyDrive/NLP project/multitool_commons/data_3.pkl")
#df=pd.read_pickle("/content/drive/MyDrive/NLP project/Abhishek (Mutli tool Chained- hotpot)/abhishek.pkl")
df=pd.read_pickle("/content/drive/MyDrive/NLP project/testing data/gemma1_commons_test.pkl")

In [10]:
df.tail()

,Q,A,C,test_prompt,sft_response
195,Greyson paid for his watch in dimes. If the co...,50,First search [number of dimes in a dollar -Wik...,### INSTRUCTION\nYour task is to generate a ch...,C: The cost of the watch in dimes is [5 / 10 =...
196,After Kaydens laptop got fully charged at 4:12...,29,First search [number of hours in a day -Wiki->...,### INSTRUCTION\nYour task is to generate a ch...,C: After Kaydens laptop got fully charged at 4...
197,Biology teacher notices that Julia is not payi...,5,First search [number of legs per insect -Wiki-...,### INSTRUCTION\nYour task is to generate a ch...,C: First search [number of legs of insects -Wi...
198,"In a farm, there are 16 animals. Some are duck...",7,First search [number of legs per horse -Wiki->...,### INSTRUCTION\nYour task is to generate a ch...,"C: In a farm, there are [16 * 2 = y1] ducks. T..."
199,Oliver reached his lab at 7 o clock in the mor...,8,First search [number of hours in a day -Wiki->...,### INSTRUCTION\nYour task is to generate a ch...,C: First search [Oliver reached his lab at 7 o...


In [11]:
df.shape

(200, 5)

In [12]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import wikipedia

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


def wikipedia_seach(query):
  language_code = 'en'

  number_of_results = 3
  headers = {
    # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
    'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
  }

  base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
  endpoint = '/search/page'
  url = base_url + language_code + endpoint

  search_query = query
  parameters = {'q': search_query, 'limit': number_of_results}
  response = requests.get(url, headers=headers, params=parameters)
  response = json.loads(response.text)

  searched_titles=[]
  for page in range(len(response['pages'])):
    display_title = response['pages'][page]['title']
    searched_titles.append(display_title)
    response['pages'][page]["article_url"] = 'http://en.wikipedia.org/?curid=' + str(response['pages'][page]['id'])
    response['pages'][page]['excerpt'] = BeautifulSoup(response['pages'][page]['excerpt']).get_text() #cleaning excerpts and saving

  excerpt_text=search_query+": "+"\n".join([page['excerpt'] for page in response['pages']]) #excerpt text

  if(len(searched_titles)!=0):
    #go to page whose title+ excerpt is most similar
    # Encode search query
    query_embedding = sentence_model.encode(search_query)
    page_texts = [page['title'] + ": " + page['excerpt'] for page in response['pages']] #page title + excerpt
    page_embeddings = sentence_model.encode(page_texts)
    # Calculate similarity scores for each page
    similarities = cosine_similarity([query_embedding], page_embeddings)[0]

    title_score_dict = dict(zip(searched_titles, similarities))

    # Get the index of the page with the highest similarity
    max_similarity_index = similarities.argmax()

    # Output the page id with the highest similarity
    max_similarity_page_id = response['pages'][max_similarity_index]['id']

    try:
      a=wikipedia.page(pageid=max_similarity_page_id)
      full_content=a.content
      return title_score_dict,excerpt_text,full_content
    except:
      return {},"",""
  else:
    return {},"",""

def QA(ques, context):
  result = question_answerer(question=ques,context=context)
  return result['answer'], result['score']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
import requests
import json
import numpy as np
from text_to_num import text2num

url = "https://google.serper.dev/search"

def google_search(query):
  payload = json.dumps({
  "q": query
})
  headers = {
  'X-API-KEY': '73bcb6abc9a600474178aa08cee6a7e32e76de6f',
  'Content-Type': 'application/json'
  }
  excerpt_text = ""
  full_content = ""
  most_similar_snippets = ""
  title_score_dict = {}


  try:
    response = requests.request("POST", url, headers=headers, data=payload)
    response = json.loads(response.text)
    #print(response)
    if 'answerBox' in response:
      if 'snippetHighlighted' in response['answerBox']:
        excerpt_text = response['answerBox']['snippetHighlighted'][0]
      elif 'answer' in response['answerBox']:
        excerpt_text = response['answerBox']['answer']
      else:
        excerpt_text = response['answerBox']['snippet']

    else:
      #print("HERE")
      full_content = [results['title'] + ": " + results['snippet'] for results in response['organic']]

    if full_content:
      query_embedding = sentence_model.encode(query)
      snippet_embeddings = sentence_model.encode(full_content)
      similarities = cosine_similarity([query_embedding], snippet_embeddings)[0]
      snippets_greater_than_70 = np.where(similarities > 0.30)[0]
      most_similar_snippets = " ".join([full_content[index] for index in snippets_greater_than_70])

    return title_score_dict,excerpt_text,most_similar_snippets
  except:
    return {},"",""

In [14]:
## CLEANING

In [ ]:
df[['question', 'answer','W']].rename(columns={'question': 'Q', 'answer': 'A'})

In [67]:
row=2
print(df.loc[row]['Q'])
print(df.loc[row]['C'])
print(df.loc[row]['A'])

What would be the length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged, if it would have been 4km longer?
First search for [Mount Panorama Circuit -Wiki-> y1]. Length of circuit is [y1 -QA(what is the length of Mount Panorama Circuit ?)-> y2]. Length after adding 4km will be [4 + y2 = y3]. The answer is y3.
10.213


### EXPERIMENT

In [15]:
from sympy import *
import re

In [16]:
def process_list_tuple(rep): ## DECISION HERE
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)


def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]
  variables['all_eqs']=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for instruction in matches:
    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      query,ans=instruction.split(" -Wiki-> ") #for in query variables
      in_query_var=re.search(r'y\d+', query)
      if(in_query_var):
        query=re.sub(r'y\d+', replace_variable, query)

      title_score_dict,excerpt_text,full_content=google_search(query)
      #forcing wiki here
      #excerpt_text=''
      #full_content=''
      if not excerpt_text and not full_content:
        print('WIKI SEARCH')
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search
        if title_score_dict: #SAVING IN VARIABLES HERE
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of wiki
          wiki_searches.append(title_score_dict)
        else:
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      else:
        variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of google

    #---QA TOOL---
    elif(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          continue

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break

      #cant send empty context to QA model
      if(context_exc!=""):
        #print(question,context_exc)
        execerpt_run=QA(question, context_exc)
        if(execerpt_run[-1]<0.75 and len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()
      else:
        execerpt_run=()
        if(len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()


      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]

    #----MATH TOOL----
    else:
      #print('going to math tool')
      #print('at math',instruction)
      variables['all_eqs'].append(instruction)
      equation_variables = re.findall(r'y\d+', instruction)
      #print('found varibales',equation_variables)
      for var in equation_variables:
        if(var in variables.keys()):
          #print('on', var)
          var_ans=process_list_tuple(variables[var])
          #print('before process',variables[var])
          #print('on process',var_ans)
          numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
          if(len(numbers)>0):
            #print('adding equation',var, numbers)
            variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
            errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
          else:
            try:
              #var_ans=str(text2num(var_ans, "en"))
              var_ans=str(w2n.word_to_num(var_ans))
              numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
              if(len(numbers)>0):
                #print('adding equation',var, numbers)
                variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
                errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
            except:
              print('var ans is not a numeric',var_ans)
              errors.append(f"Error: '{var_ans}' not numeric")


  return variables,wiki_searches,errors


In [17]:
row=1
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['sft_response']
matches = re.findall(r'\[(.*?)\]', text)
matches

In a parking lot, There are 10 cars and 2 bikes. Find out the number of wheels in that parking lot.
First search [car number of wheels -Wiki-> y1]. A car has [y1 -QA(How many wheels does a car have?)-> y2] wheels each. Then search [bike number of wheels -Wiki-> y3]. A bike has [y3 -QA(How many wheels does a bike have?)-> y4] wheels each. So, there are [10*y2 + 2*y4 = y5] wheels in total.
44


['10 + 2 = y1',
 'number of wheels in a parking lot -Wiki-> y1',
 'y1 -QA(How many wheels in a parking lot?)-> y2']

In [18]:
v,w,e=get_variables(text)

WIKI SEARCH


In [ ]:
v

In [21]:
solve_all(v['all_eqs'])

{'y1': 12}

In [22]:
def solve_all(equations):
  ans_dict={}
  list_eq=[]
  for i in equations:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)
  #print(list_eq)
  result =solve(list_eq)
  #print(result)
  if(str(type(result))=="<class 'list'>"):
    result=result[0]
  for i in result.keys():
      ans_dict[str(i)]=result[i]
  return ans_dict

In [23]:
processed=[]

In [24]:

for row in tqdm(range(df.shape[0])):
  row_d=df.iloc[row].to_dict()
  try:
    answer_var=row_d['sft_response'].lower().split("answer is")[-1].replace(".","").strip()
    variables, wiki_searches, errors = get_variables(row_d['C'])
    if(len(variables['all_eqs'])>0):
      #solve eqs
      try:
        variables.update(solve_all(variables['all_eqs']))
      except:
        errors.append("Equaltions not solvable")
    if(answer_var in variables.keys()):
      answer=variables[answer_var]
    else:
      answer="not found"
  except:
    variables=[]
    wiki_searches=[]
    errors=['Outer Error Handling']
    answer="not found"

  row_d['Variables'] = variables
  row_d['Searches'] = wiki_searches
  row_d['Errors'] = errors
  row_d['Generated_answer'] = answer
  processed.append(row_d)

  # Save to pickle file every 50 rows
  if row % 50 == 0 and row != 0:
      processed_df = pd.DataFrame(processed)
      #processed_df.to_pickle(f'processed_rows_wiki2.pkl')


# Save any remaining processed rows to a pickle file
if(processed):
    processed_df = pd.DataFrame(processed)
    #processed_df.to_pickle(f'processed_rows_final_wiki2.pkl')

  0%|          | 0/200 [00:00<?, ?it/s]

WIKI SEARCH
WIKI SEARCH
var ans is not a numeric duck number of legs
var ans is not a numeric horse number of legs


  0%|          | 1/200 [00:29<1:36:12, 29.01s/it]

WIKI SEARCH
WIKI SEARCH


  1%|          | 2/200 [00:39<59:51, 18.14s/it]  

...

 99%|█████████▉| 198/200 [53:56<00:42, 21.44s/it]

WIKI SEARCH
WIKI SEARCH


100%|█████████▉| 199/200 [54:25<00:23, 23.57s/it]

var ans is not a numeric orange
WIKI SEARCH


100%|██████████| 200/200 [54:32<00:00, 16.36s/it]


In [26]:
processed_df.shape

(200, 9)

In [27]:
row=2
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['sft_response']
matches = re.findall(r'\[(.*?)\]', text)
matches

If fencing was done around a park in the shape of a square. The total cost of fencing was 172 dollars. Find the cost of fencing each side.
First search [number of sides in a square -Wiki-> y1]. A square has [y1 -QA(How many sides does a square have?)-> y2] sides. Let the cost of fencing each side be x dollars. So, the total cost of fencing is [y2*x = 172]. Solve this equation to find the cost of fencing each side.
43


['172 / 4 = y1', 'y1 = 44 dollars', 'y1 * 2 = y2']

In [29]:
processed_df.iloc[2]['Variables']

{'all_eqs': ['y2*x = 172', 'y2 = 4'], 'y1': ['4 sides', ''], 'y2': 4, 'x': 43}

In [28]:
def check_answer(row):
  gen=row['Generated_answer']
  act=row['A']
  all_vars=row['Variables']
  #print(all_vars)

  if(gen==act):
    return True
  if(str(gen).find(str(act))!=-1):
    return True
  if(str(type(all_vars))=="<class 'dict'>"):
    for v in all_vars.values():
      if(str(v)==str(act)):
        return True

  return False

# Apply the function to each row
processed_df['is_final_in_solved'] = processed_df.apply(check_answer, axis=1)

In [29]:
processed_df['is_final_in_solved'].unique()

array([False,  True])

In [30]:
# Count the rows where final_answer is present in solved_answer list
count_valid = processed_df[processed_df['is_final_in_solved']].shape[0]

In [31]:
count_valid

38

In [35]:
processed_df.shape

(200, 10)

In [46]:
#50%

In [34]:
processed_df.head()

,Q,A,C,test_prompt,sft_response,Variables,Searches,Errors,Generated_answer,is_final_in_solved
0,Harper decides to go to a nearby animal farm t...,7,First search [duck number of legs -Wiki-> y1]....,### INSTRUCTION\nYour task is to generate a ch...,C: The total number of animals in the farm is ...,"{'all_eqs': ['x*y2 + y*y4 = 30', 'y2 = 2', 'y4...",[],"[WARNING: '['2']'numbers in var ans, WARNING: ...",not found,True
1,"In a parking lot, There are 10 cars and 2 bike...",44,First search [car number of wheels -Wiki-> y1]...,### INSTRUCTION\nYour task is to generate a ch...,C: First search [number of wheels in a car -Wi...,"{'all_eqs': ['10*y2 + 2*y4 = y5', 'y2 = 7', 'y...",[],"[WARNING: '['7']'numbers in var ans, WARNING: ...",not found,False
2,If fencing was done around a park in the shape...,43,First search [number of sides in a square -Wik...,### INSTRUCTION\nYour task is to generate a ch...,"C: If the park is in the shape of a square, al...","{'all_eqs': ['y2*x = 172', 'y2 = 4'], 'y1': ['...",[],[WARNING: '['4']'numbers in var ans],not found,True
3,"Alexander goes to school at 12 pm. On Tuesday,...",4,First search [number of hours in a day -Wiki->...,### INSTRUCTION\nYour task is to generate a ch...,C: Alexander goes to school at [12 = y1] pm. H...,"{'all_eqs': ['12 + y4 = y5', 'y4 = 4'], 'y1': ...",[],[WARNING: '['4']'numbers in var ans],16,True
4,"At the end of the month, Hudson was checking h...",15,First search [number of days in December -Wiki...,### INSTRUCTION\nYour task is to generate a ch...,C: First search [number of burgers Hudson eats...,"{'all_eqs': ['x*y2 = 465', 'y2 = 31'], 'y1': [...",[],[WARNING: '['31']'numbers in var ans],not found,True


In [84]:
processed_df.to_pickle("/content/drive/MyDrive/NLP project/multitool_commons/gpt_forced_wiki_common_abhishek_validated.pkl")

In [37]:
processed_df.to_csv('/content/drive/MyDrive/NLP project/testing data/mistral1_commons_test_processed.csv')